## Load Libraries

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Define Transformations

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

## Load the Data

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

train_data = datasets.CIFAR10( root="./data", train=True, download=True, transform = transform)
test_data = datasets.CIFAR10( root="./data", train=False, download=True, transform = transform)

100.0%


In [4]:
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

## Build the Architecture

In [5]:
class cnn_cifar(nn.Module):
    def __init__(self):
        super(cnn_cifar, self).__init__()
        self.cnn1 = nn.Conv2d(3, 6, 5)
        self.cnn2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc1 = nn.Linear(120, 84)
        self.fc1 = nn.Linear(84, 10)
        
        def forward(self, x):
            x=self.cnn1(x)
            x=torch.relu(x)
            x=torch.max_pool2d(x, 2)

            x=self.cnn2(x)
            x=torch.relu(x)
            x=torch.max_pool2d(x, 2)

            x=torch.flatten(x, -1)

            x=self.fc1(x)
            x=torch.relu(x)

            x=self.fc2(x)
            x=torch.relu(x)

            x=self.fc3(x)

            return torch.relu(x)

In [9]:
model = cnn_cifar()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

##  Training

In [ ]:
for epoch in range(10):
    for images, label in enumerate(train_loader):
        